In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# run data_processing.ipynb to generate the train, val, test sets here
# load and split train, val, test, (X, y)
TRAIN_DATA = 'data/training.1440000.csv'
VAL_DATA = 'data/validation.80000.csv'
TEST_DATA = 'data/test.80000.csv'

df_train = pd.read_csv(TRAIN_DATA)
df_val = pd.read_csv(VAL_DATA)
df_test = pd.read_csv(TEST_DATA)

X_train, y_train = df_train['text'], df_train['target']
X_val, y_val = df_val['text'], df_val['target']
X_test, y_test = df_test['text'], df_test['target']

In [4]:
!pip install tensorflow

     |████████████████████████████████| 228.5 MB 30 kB/s s eta 0:00:01   |█▍                              | 10.1 MB 3.2 MB/s eta 0:01:09
     |████████████████████████████████| 1.6 MB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 123 kB 17.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 17.6 MB/s eta 0:00:01
     |████████████████████████████████| 13.2 MB 19.3 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 4.3 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 7.5 MB/s  eta 0:00:01
     |████████████████████████████████| 4.4 MB 53.9 MB/s eta 0:00:01
     |████████████████████████████████| 961 kB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 438 kB 10.8 MB/s eta 0:00:01
  Using cached termcolor-1.1.0-py3-none-any.whl
     |████████████████████████████████| 5.8 MB 3.4 MB/s eta 0:00:011
  Using cached tensorboard_data_server-0.6.1-py3-none-maco

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_words=10000
tokenizer=Tokenizer(max_words)
tokenizer.fit_on_texts(X_train)
sequence_train=tokenizer.texts_to_sequences(X_train)
sequence_val=tokenizer.texts_to_sequences(X_val)
sequence_test=tokenizer.texts_to_sequences(X_test)

In [6]:
word2vec=tokenizer.word_index
V=len(word2vec)
print('dataset has %s number of independent tokens' %V)

dataset has 277457 number of independent tokens


In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
data_train=pad_sequences(sequence_train)
data_train.shape

(1440000, 116)

In [8]:
T=data_train.shape[1]
data_test=pad_sequences(sequence_test,maxlen=T)
data_test.shape
data_val=pad_sequences(sequence_val,maxlen=T)
data_val.shape

(80000, 116)

In [9]:
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding
from tensorflow.keras.models import Model

In [10]:
D=20
i=Input((T,))
x=Embedding(V+1,D)(i)
x=Conv1D(32,3,activation='relu')(x)
x=MaxPooling1D(3)(x)
x=Conv1D(64,3,activation='relu')(x)
x=MaxPooling1D(3)(x)
x=Conv1D(128,3,activation='relu')(x)
x=GlobalMaxPooling1D()(x)
x=Dense(5,activation='softmax')(x)
model=Model(i,x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 116)]             0         
                                                                 
 embedding (Embedding)       (None, 116, 20)           5549160   
                                                                 
 conv1d (Conv1D)             (None, 114, 32)           1952      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 38, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 36, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 12, 64)           0         
 1D)                                                         

2022-06-06 15:00:09.612205: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn_senti=model.fit(data_train,y_train,validation_data=(data_val,y_val),epochs=5,batch_size=100)

Epoch 1/5
14400/14400 [==============================] - 623s 43ms/step - loss: 0.4187 - accuracy: 0.8058 - val_loss: 0.3949 - val_accuracy: 0.8206
Epoch 2/5
14400/14400 [==============================] - 615s 43ms/step - loss: 0.3803 - accuracy: 0.8287 - val_loss: 0.3851 - val_accuracy: 0.8265
Epoch 3/5
14400/14400 [==============================] - 618s 43ms/step - loss: 0.3659 - accuracy: 0.8367 - val_loss: 0.3885 - val_accuracy: 0.8257
Epoch 4/5
14400/14400 [==============================] - 606s 42ms/step - loss: 0.3558 - accuracy: 0.8422 - val_loss: 0.3879 - val_accuracy: 0.8253
Epoch 5/5
14400/14400 [==============================] - 613s 43ms/step - loss: 0.3480 - accuracy: 0.8463 - val_loss: 0.3893 - val_accuracy: 0.8239


In [14]:
y_pred=model.predict(data_test)
y_pred=np.argmax(y_pred,axis=1)
y_pred


2500/2500 [==============================] - 3s 1ms/step


array([1, 1, 1, ..., 0, 0, 0])

In [15]:
pd.DataFrame(y_pred).to_csv('predictions/cnn.pred.80000.csv', index=False)

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83     40146
           1       0.84      0.82      0.83     39854

    accuracy                           0.83     80000
   macro avg       0.83      0.83      0.83     80000
weighted avg       0.83      0.83      0.83     80000

